## Bagit package duplication

Deduplication with digital material is usually considered at a file level - ie. this file is identical to that file, these files have identical titles, these files are similar.

The Greer Archive contains 421 disk images of various forms of media, all with varying levels of duplication. Duplicate files within a digital collection are not necessarily a problem, as duplicates can mean different things in different contexts. Consider duplicate powerpoint presentations on media labelled with the event at which the talk was delivered. In an analogue archive, we would rarely (if ever) weed documents duplicated within items, thoughwe would base decisions on acquisition around the uniqueness of material at appraisal.

However analysis of duplication at a file level indicates under a third of the files recovered are unique. This notebook is an attempt to map similarities of groupings of files. It does this by comparing the checksums for the contents of each package (in a bagit structure) and then mapping the similarity.

First we'll need a list of the bags. 

In [1]:
import os
import bagit
import difflib

directory = 'corpus'
#if using locally, change this to your base bag directory
baglist = []
for root, _, files in os.walk(directory):
    if 'bagit.txt' in files:
        baglist.append(bagit.Bag(root))

We'll define some functions to help us do the comparisons.

In [2]:
def filter_hashes(p_entries, alg, filter=None):
    '''
    Filter hashes to a specific directory.
    Intended for use with standard package structures.
    '''
    h = []
    for file, hashes in p_entries.items():
        if filter is not None:
            if file.startswith(os.path.normpath(filter)):
                h.append(hashes.get(alg))
        else:
            h.append(hashes.get(alg))
    return(h)

In [3]:
def rehash(bag, alg, filter=None):
    'generates a hash list if the two bags do not have matching algs'
    hashes = []
    os.chdir(bag.path)
    for root, _, files in os.walk(os.getcwd):
        for file in files:
            p = os.path.relpath(os.path.join(root, file))
            if filter is not None:
                if p.startswith(filter):
                    h = bagit.generate_manifest_lines(p, [alg])
                    hashes.append(h[1])
            else:
                h = bagit.generate_manifest_lines(p, [alg])
                hashes.append(h[1])
    return(hashes)

In [4]:
def compare_bags(a, b, filter=None):
    algorithm = None
    for alg in a.algorithms:
        if alg in b.algorithms:
            algorithm = alg
    a_hashes = filter_hashes(a.payload_entries(), algorithm, filter)
    if algorithm is None:
        b_hashes = rehash(b, a.algorithms[0], filter=filter)
    b_hashes = filter_hashes(b.payload_entries(), algorithm, filter)
    if a_hashes != [] and b_hashes != []:
        s = difflib.SequenceMatcher(None, a_hashes, b_hashes)
        return(s)

Now we're going to construct two network graphs using the networkx library. For the first graph, we're using difflib to compare the sequences of hashes and derive a ratio of similarity. A ratio of 0.0 means the bags are in no way similar and 1.0 means they are exactly the same. In the graph, each bag is a node with edges between when similarity is over 0.5. Note the compare_bags function has an optional filter keyword, if you want to filter to just a single directory within the bags.

For the second graph, we're drawing edges between packages where one is completely duplicated within the other. 

We're also going to filter in some descriptive metadata from the bag-info.txt files, so we can interpret the visualised graph better. If applying this to your own sets of bags you may need to adjust the metadata and the filtered directory.

In [5]:
import networkx as nx
graph_one = nx.MultiDiGraph()
graph_two = nx.DiGraph()
for bag_a in baglist:
    name_a = os.path.split(bag_a.path)[1]
    #adjust your bag metadata here if needed, or remove the kw variable altogether.
    kw = {'carrier': bag_a.info.get('carrier'), 'title': bag_a.info.get('title')}
    graph_one.add_node(name_a, **kw)
    graph_two.add_node(name_a, **kw)
    for bag_b in baglist:
        name_b = os.path.split(bag_b.path)[1]
        if bag_a.path != bag_b.path and (name_a, name_b) not in graph_one.edges:
            s = compare_bags(bag_a, bag_b)
            if s is not None:
                score = s.quick_ratio()
                if score > 0.5:
                    graph_one.add_edge(name_a, name_b, weight=score)
                blocks = s.get_matching_blocks()
                matches = 0
                len_a = blocks[-1][0]
                for block in blocks:
                    matches += block[2]
                if len_a == matches:
                    graph_two.add_edge(name_a, name_b)
                

Here we're going to construct a searchable index, where you can pick a package and it will tell you about similar packages.

In [6]:
import pandas as pd
from ipywidgets import interact
f = nx.to_pandas_edgelist(graph_one)
st = [graph_one.nodes.data()[key]['title'] for key in f['source']]
tt = [graph_one.nodes.data()[key]['title'] for key in f['target']]
data = {'source_title': st, 'target_title': tt}
f = f.join(pd.DataFrame(data=data))
def filter(ind):
    f_slice = f[f['source'] == ind]
    return f_slice
ind = f['source'].drop_duplicates().sort_values()
interact(filter, ind=ind)

interactive(children=(Dropdown(description='ind', options=('sip_2014.0037.00600', 'sip_2014.0037.00604', 'sip_…

<function __main__.filter(ind)>

Finally, we're going to visualise the graphs using Bokeh (I tried this with Plot.ly, but it doesn't visualise network graphs as nicely). Darker edges are where packages are more similar. Select a node to bold connected edges.

In [7]:
from bokeh.io import show, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, TapTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import magma, viridis

#colour coding edges by weight
edge_attrs = {}
cp = magma(5)
cp.reverse()
for start_node, end_node, key, data in graph_one.edges(data=True, keys=True):
    w = round(data['weight']*10)-5
    edge_attrs[(start_node, end_node, key)] = cp[w-1]
nx.set_edge_attributes(graph_one, edge_attrs, name="edge_color")

#colour coding nodes by carrier type
node_attrs = {}
c = []
for node, data in graph_one.nodes.data():
    c.append(data['carrier'])
c = set(c)
m = viridis(len(c))
colours = dict(zip(c, m))
colourmap = {}
for node, data in graph_one.nodes.data():
    node_attrs.update({node: colours[data['carrier']]})
nx.set_node_attributes(graph_one, node_attrs, name="node_color")

#building the graph
output_notebook(notebook_type='jupyter')
plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Bag similarity"

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("title", "@title"), ("carrier", "@carrier")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), TapTool())

graph_renderer = from_networkx(graph_one, nx.spring_layout, scale=1, center=(0, 0), weight="weight")
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width="weight", line_color="edge_color")
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_width=5, line_color="edge_color")
graph_renderer.selection_policy = NodesAndLinkedEdges()
plot.renderers.append(graph_renderer)

show(plot)

Loading BokehJS ...

In [8]:
node_attrs = {}
c = []
for node, data in graph_two.nodes.data():
    c.append(data['carrier'])
c = set(c)
m = magma(len(c))
colours = dict(zip(c, m))
colourmap = {}
for node, data in graph_two.nodes.data():
    node_attrs.update({node: colours[data['carrier']]})
nx.set_node_attributes(graph_two, node_attrs, name="node_color")
    
output_notebook(notebook_type='jupyter')
plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Bag duplication"

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("title", "@title"), ("carrier", "@carrier")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), TapTool())

graph_renderer = from_networkx(graph_two, nx.spring_layout, scale=1, center=(0, 0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_width=5, line_color='red')
graph_renderer.selection_policy = NodesAndLinkedEdges()
plot.renderers.append(graph_renderer)

show(plot)

Loading BokehJS ...

Homework - You could equally apply this approach to filenames to find semantically similar packages!

The below is less useful, but fun anyway - creates network graphs for similar bags where each node is a file, drawing edges between exact duplicates. Actually useful only with small pac

In [12]:
def filter_bags(bagname):
    graph = nx.Graph()
    f_filtered = f[f['source'] == bagname]
    f_filtered = f_filtered['target'].tolist()
    f_filtered.append(bagname)
    m = viridis(len(f_filtered))
    colours = dict(zip(f_filtered, m))
    filtered_bags = [bag for bag in baglist if os.path.split(bag.path)[1] in f_filtered]
    shell_layout = [[os.path.join(bag.path, entry) for entry in bag.payload_entries()] for bag in filtered_bags]
    for bag in filtered_bags:
        bagname = os.path.split(bag.path)[1]
        for file, hashes in bag.payload_entries().items():
            f_path = os.path.join(bag.path, file)
            graph.add_node(f_path, **hashes, bag=bagname, colour=colours[bagname])
            if hashes.get('sha256') in nx.get_node_attributes(graph, 'sha256').values():
                linked_nodes = [k for k, v in nx.get_node_attributes(graph, 'sha256').items() if hashes.get('sha256') == v]
                for node in linked_nodes:
                    graph.add_edge(f_path, node)
    plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("bag", "@bag"), ("sha256", "@sha256")])
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), TapTool())
    graph_renderer = from_networkx(graph, nx.shell_layout, nlist=shell_layout, scale=1, center=(0, 0))
    graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='colour')
    graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=2)
    graph_renderer.edge_renderer.selection_glyph = MultiLine(line_width=5, line_color='red',)
    graph_renderer.selection_policy = NodesAndLinkedEdges()
    plot.renderers.append(graph_renderer)
    show(plot)

interact(filter_bags, bagname=f['source'].drop_duplicates().sort_values())

interactive(children=(Dropdown(description='bagname', options=('sip_2014.0037.00600', 'sip_2014.0037.00604', '…

<function __main__.filter_bags(bagname)>